In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
from torch import nn
import numpy as np
from PIL import Image
import random 
import os 
device = 'cuda'

In [4]:
# transform stuff to test
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(256, 256)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

In [7]:
from torchvision import datasets
train_data = datasets.ImageFolder(root='datasets',transform=data_transform)
test_data = datasets.ImageFolder(root='test',transform=data_transform)

In [8]:
#Get class names
class_names = train_data.classes
class_names
#Get class dict
class_dict = train_data.class_to_idx
class_dict

{'Atypical': 0, 'Indeterminate': 1, 'Negative': 2, 'Typical': 3}

In [10]:

#setup dataloader 
BATCH_SIZE = 16
NUM_WORKERS = os.cpu_count()-4

train_dataloader_simple = DataLoader(train_data, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(test_data, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False, 
                                    num_workers=NUM_WORKERS)

In [11]:
class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*64*64,
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        # print(x.shape)
        x = self.conv_block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  output_shape=len(train_data.classes)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=4, bias=True)
  )
)

In [50]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader_simple))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))
    
# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 256, 256])

Output logits:
tensor([[0.0329, 0.0068, 0.0042, 0.0352]], device='cuda:0')

Output prediction probabilities:
tensor([[0.2533, 0.2467, 0.2461, 0.2539]], device='cuda:0')

Output prediction label:
tensor([3], device='cuda:0')

Actual label:
0


In [35]:

#print(f"Output logits:\n{pred}\n")
#print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Output prediction label:
tensor([3], device='cuda:0')

Actual label:
2


In [85]:
from torchinfo import summary 
summary(model_0, input_size=[1, 3, 256, 256]) # do a test pass through of an example input size


Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 4]                    --
├─Sequential: 1-1                        [1, 10, 128, 128]         --
│    └─Conv2d: 2-1                       [1, 10, 256, 256]         280
│    └─ReLU: 2-2                         [1, 10, 256, 256]         --
│    └─Conv2d: 2-3                       [1, 10, 256, 256]         910
│    └─ReLU: 2-4                         [1, 10, 256, 256]         --
│    └─MaxPool2d: 2-5                    [1, 10, 128, 128]         --
├─Sequential: 1-2                        [1, 10, 64, 64]           --
│    └─Conv2d: 2-6                       [1, 10, 128, 128]         910
│    └─ReLU: 2-7                         [1, 10, 128, 128]         --
│    └─Conv2d: 2-8                       [1, 10, 128, 128]         910
│    └─ReLU: 2-9                         [1, 10, 128, 128]         --
│    └─MaxPool2d: 2-10                   [1, 10, 64, 64]           --
├─Sequentia

In [60]:
!conda install -c conda-forge torchinfo -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\thanawit\.conda\envs\twin-pseudo

  added / updated specs:
    - torchinfo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       h56e8100_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    torchinfo-1.8.0            |     pyhd8ed1ab_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         318 KB

The following NEW packages will be INSTALLED:

  torchinfo          conda-forge/noarch::torchinfo-1.8.0-pyhd8ed1ab_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2023.01.10~ --> conda-forge::ca-certificates-2023.5.7-h56e8100_0
  certifi            pkgs/main/win-64::certifi-2



==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [88]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
summary(model, input_size=[1, 3, 256, 256]) # do a test pass through of an example input size


Using cache found in C:\Users\thanawit/.cache\torch\hub\pytorch_vision_v0.10.0


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 128, 128]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 128, 128]         128
├─ReLU: 1-3                              [1, 64, 128, 128]         --
├─MaxPool2d: 1-4                         [1, 64, 64, 64]           --
├─Sequential: 1-5                        [1, 64, 64, 64]           --
│    └─BasicBlock: 2-1                   [1, 64, 64, 64]           --
│    │    └─Conv2d: 3-1                  [1, 64, 64, 64]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 64, 64]           128
│    │    └─ReLU: 3-3                    [1, 64, 64, 64]           --
│    │    └─Conv2d: 3-4                  [1, 64, 64, 64]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 64, 64]           128
│    │    └─ReLU: 3-6                    [1, 64, 64, 64]           --
│